### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [4]:
!pip install langchain

In [5]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")
openai.base_url=os.getenv("OPENAI_API_BASE")

groq_api_key=os.getenv("GROQ_API_KEY")

In [6]:
!pip install langchain_groq

In [11]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C86D433730>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C86D4336A0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
!pip install langchain_core

In [12]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [13]:
result

AIMessage(content='Bonjour Comment allez-vous ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 47, 'total_tokens': 54, 'completion_time': 0.018248114, 'prompt_time': 0.002294094, 'queue_time': 0.051027866, 'total_time': 0.020542208}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0a8a8d8d-7392-447c-9144-c9a126851a2e-0', usage_metadata={'input_tokens': 47, 'output_tokens': 7, 'total_tokens': 54})

In [14]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour Comment allez-vous ?'

In [16]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour Comment allez-vous ?'

In [17]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [18]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [19]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [23]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"french","text":"hello"})

'Bonjour.'